# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# TD1 - Introduction à Julia

Ce TD constitue une introduction au langage Julia et au calepin Jupyter. L'Exercice 1 a pour but d'introduire les commandes de base de Julia et de la librairie *Distributions* tandis que l'Exercice 2 a pour but d'introduire les DataFrames et la puissante librairie *Gadfly*, similaire à *ggplot2* en *R*, pour la production de graphiques.

# Exercice 1

Dans cet exercice, nous nous intéressons au problèmes du nombre d'achats requis pour compléter une collection de $n$ items distincts où à chaque achat, la probabilité d'obtenir un item particulier est de $1/n$. En classe, nous avons considéré ce problème pour les 31 cartes des capitaines de la LNH contenues dans des boîtes de céréales.

Voici les différentes étapes qui nous permettrons d'étudier le problème :

1. Simulation du nombre d'essais requis pour obtenir la $i^e$ nouvel item de la collection.
2. Simulation du nombre d'essais requis pour obtenir la collection complète.
3. Affichage du nombre d'essais requis pour 10 000 essais afin d'obtenir la collection complète.
4. Estimation de la moyenne du nombre d'essais requis pour obtenir la collection complète.
5. Estimation de la probabilité que 100 essais soit suffisants pour obtenir la collection complète.

Avant tout, chargeons les librairies requises.

In [ ]:
using Gadfly, Statistics, Distributions

## 1. Simulation du nombre d'essais requis pour obtenir le $i^e$ nouvel item

Une fois le code permettant de simuler le nombre d'essais requis pour obtenir la $i^e$ nouvelle carte, transformez-le en fonction nommée *ntrial* qui prend en argument $i$ et la taille de collection $n$.

In [ ]:
n = 31 # Nombre d'items distincs de la collections
i = 31

pᵢ = (n-i+1)/n # probabilité de succès
Tᵢ = Geometric(pᵢ) # Création d'un objet «Loi Géométrique de paramètre p»
tᵢ = 1 + rand(Tᵢ) # Application de la méthode rand à l'objet

#= Ici on ajoute 1 car la loi géométrique est paramétrée de façon non-standard
dans la librairie Distributions.jl. Au lieu de dénombrer le nombre d'essais 
nécessaires pour obtenir le premier succès, on obtient le nombre d'échecs avant
le premier succès. =#

println("La réalisation de T est $tᵢ")


In [ ]:
function ntrial(i::Int,n::Int)
   @assert 0<i<=n # Vérification de la validité de i (facultatif)
    
    if i>1
        pᵢ = (n-i+1)/n # probabilité de succès
        Tᵢ = Geometric(pᵢ) # Création d'un objet «Loi Géométrique de paramètre p»
        tᵢ = 1 + rand(Tᵢ) # Application de la méthode rand à l'objet
    else
        tᵢ = 1
    end
    
    return tᵢ
end

In [ ]:
tᵢ = ntrial(25,31)

In [ ]:
# Pour le plaisir, on peut tracer la taille de la collection en fonction du nombre d'essais
# CECI N'EST PAS L'HISTOGRAMME

y = 1:n
t = ntrial.(y,n)
cs = cumsum(t)
plot(x=cs, y=y, Geom.step)

## 2. Simulation du nombre d'achats requis pour obtenir la collection complète.

Vous devez ici simuler un suite d'essais nécessaires pour obtenir la collection complète. Une fois votre code complété, transformez-le en fonction nommé *ntotal* qui prend en argument la taille de la collection.

In [ ]:
s = 0
for i=1:n
    s += ntrial(i,n)
end

println("Le nombre d'essais total est de $s")

In [ ]:
function ntotal(n::Int)
   @assert n>0
    
    s = 0
    for i=1:n
        s += ntrial(i,n)
    end
    
    return s
end

In [ ]:
ntotal(31)

## 3. Affichage du nombre d'essais requis pour 10 000 cas.

Il faut avant tout simuler le nombre d'essais requis pour compléter la collection 10 000 fois. Par la suite, afficher le nombre d'essais requis avec un histogramme à 100 classes avec l'argument *Geom.histogram(bincount=100)* de la fonction *plot* de la librairie *Gadfly*.

In [ ]:
# Génération des 10 000 cas
nSim = 10000
s = Int64[]

for j=1:nSim
    push!( s, ntotal(31) )
end

In [ ]:
plot(x=s, Geom.histogram(bincount=100))

## 4. Estimation de la moyenne du nombre d'essais requis pour obtenir la collection complète.

À l'aide des 10 000 réalisations, calculez la moyenne empirique de nombre d'essais requis pour obtenir la collection complète de $n=31$ items. Comment elle se compare à la moyenne calculée en classe de 124 essais requis ?

In [ ]:
x̄ = mean(s)
println("La moyenne empirique est de $x̄ essais")

## 5. Estimation de la probabilité que 100 essais soit suffisants pour obtenir la collection complète.

Pour ce faire, calculez la proportion empirique des réalisations parmi les 10 000 simulées où 100 essais ou moins ont été suffisants pour compléter la collection.

In [ ]:
α = count(s .< 100) / length(s)

println("La probabilité empirique que ça prenne moins de 100 essais est de $α")

# Exercice 2

Dans cet exercice, vous devrez charger les données météorologiques enregistrées à l'Aéroport Pierre-Elliot-Trudeau à Montréal et afficher les températures annuelles maximales. Les étapes sont les suivantes :

1. Chargement du fichier de données.
2. Récupération des années complètes uniquement.
3. Extraction des températures annuelles maximales.
4. Affichage des températures annuelles maximales en fonction des années.

Avant tout, chargeons les librairies requises.


In [ ]:
using DataFrames, CSV, Gadfly

## 1. Chargement du fichier de données

Le chargement du fichier de données se fait avec la fonction *CSV.read* de la librairie *CSV*. La variable de sortie sera un DataFrame.

In [ ]:
data = CSV.read("meteoMontreal.csv")
first(data,5) #Permet d'afficher uniquement les 5 premières lignes du DataFrame

## 2. Récupération des années complètes

Pour que les maxima annuels aient du sens, il faut qu'ils soient pris sur l'année complète. Par exemple, on ne veut pas récupérer le maximum annuel d'une année où l'été n'a pas été enregistré. Pour la station en question, il faut retirer les années 1941 et 2013.

In [ ]:
ind = (data[!,:LOCAL_YEAR] .> 1941) .& (data[!,:LOCAL_YEAR] .< 2013)
data = data[ind, :]
first(data,5)

## 3.  Extraction des températures maximales annuelles

Il y a plusieurs façons de procéder. Je vous propose d'utiliser la fonction *by()* de la librairie *DataFrame* pour manipuler aisément les données dans un DataFrame.

Vous devrez également vous soucier des valeurs manquantes. Ici, je propose d'utiliser la fonction *skipmissing()* pour ignorer naïvement les valeurs manquantes.

In [ ]:
M =  by(data, :LOCAL_YEAR, data -> maximum(skipmissing(data[:, :MAX_TEMPERATURE])))
first(M,5)

In [ ]:
# Ici, je propose de modifier les noms des colonnes pour un affichage plus élégant
rename!(M, :LOCAL_YEAR =>:Year, :x1 => :Maximum)
first(M,5)

## 4. Affichage des températures annuelles maximales en fonction des années.

Lorsque les données à afficher sont dans un DataFrame, le traçage des graphiques est simple avec Gadfly. Il suffit de mentionner quelle colonne est le $x$ et quelle colonne est le $y$.

In [ ]:
plot(M, x=:Year, y=:Maximum, Geom.point, Geom.line)